**Import Libraries**

In [0]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import time
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from google.colab import files
# https://pypi.python.org/pypi/pydot
#!apt-get -qq install -y graphviz && pip install -q pydot

**Support Functions**

In [0]:
def get_emotion(ohv):
    indx = np.argmax(ohv)
    # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral    
    if indx == 0:
        return 'angry'
    elif indx == 1:
        return 'disgust'
    elif indx == 2:
        return 'fear'
    elif indx == 3:
        return 'happy'
    elif indx == 4:
        return 'sad'
    elif indx == 5:
        return 'surprise'
    elif indx == 6:
        return 'neutral'

In [0]:
def test_model(model, testingset, x_testing, y_testing):
  score = model.evaluate(x_testing, y_testing)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1] * 100)
  
  img_indx = np.uint32(np.random.rand()*(testingset.shape[0] - 1))
  sample = x_testing[img_indx, :]
  sample = sample.reshape(img_dim, img_dim)

  pred_cls = model.predict(sample.reshape(1, img_dim, img_dim, 1))

  plt.imshow(sample, cmap='gray')
  plt.show()
  print('> testing image index: %d\n> true emotion: %s\n> predicted emotion: %s'
        % (img_indx, get_emotion(y_testing[img_indx, :]), get_emotion(pred_cls)))

**Data Preparing**

In [0]:
files.upload()

trainingset = np.loadtxt('fer2013_training.csv', delimiter=',')
pub_testingset = np.loadtxt('fer2013_public_test.csv', delimiter=',')
priv_testingset = np.loadtxt('fer2013_private_test.csv', delimiter=',')
trainingset

In [0]:
# Get data and labels
n_inputs = 2304 # 48x48 pixels
n_classes = 7 # angry, disgust, fear, happy, neutral, sad, and surprise
img_dim = 48

x_training = trainingset[:, 0:n_inputs]
y_training = trainingset[:, n_inputs:n_inputs + n_classes]

x_pub_testing = pub_testingset[:, 0:n_inputs]
y_pub_testing = pub_testingset[:, n_inputs:n_inputs + n_classes]

x_priv_testing = priv_testingset[:, 0:n_inputs]
y_priv_testing = priv_testingset[:, n_inputs:n_inputs + n_classes]

# Reshape
x_training = x_training.reshape(x_training.shape[0], img_dim, img_dim)
x_training = np.expand_dims(x_training, axis=4)

x_pub_testing = x_pub_testing.reshape(x_pub_testing.shape[0], img_dim, img_dim)
x_pub_testing = np.expand_dims(x_pub_testing, axis=4)

x_priv_testing = x_priv_testing.reshape(x_priv_testing.shape[0], img_dim, img_dim)
x_priv_testing = np.expand_dims(x_priv_testing, axis=4)

**Build the Model**

In [0]:
# 48x48 portrait image
input_image = Input(shape=(48, 48, 1), name='Input')

# conv, pooling layers + dropout
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_1')(input_image)
x = Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_2')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_3')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_4')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_5')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_6')(x)
x = MaxPooling2D(pool_size=(2, 2), name='pooling1')(x)
x = Dropout(rate=0.5, name='conv_dropout1')(x)

x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_1')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_2')(x)
x = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_3')(x)
x = Conv2D(filters=256, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_4')(x)
x = Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_5')(x)
x = Conv2D(filters=512, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_6')(x)
x = MaxPooling2D(pool_size=(2, 2), name='pooling2')(x)
x = Dropout(rate=0.5, name='conv_dropout2')(x)

# matrix ---> vector
x = Flatten(name='flatten')(x)

# FC layers + dropout
x = Dense(units=2048, activation='relu', name='fc1')(x)
x = Dropout(rate=0.5, name='fc_dropout1')(x)

x = Dense(units=2048, activation='relu', name='fc2')(x)
x = Dropout(rate=0.5, name='fc_dropout2')(x)

output_label = Dense(units=n_classes, activation='softmax', name='Output')(x)

# define model
model = Model(inputs=input_image, outputs=output_label, name='fer_cnn')

# print model summary
model.summary()

In [0]:
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

** Training and Evaluating the Model **

In [0]:
# declare learning rate, loss function, and model metric
loss = 'categorical_crossentropy'
lr = 0.0001
model.compile(loss=loss, optimizer=Adam(lr=lr), metrics=['accuracy'])

# train the model
batch_size = 128
epochs = 30

starting_time = time.time()
history = model.fit(x_training, y_training,
                    batch_size,
                    validation_data=(x_pub_testing, y_pub_testing),
                    epochs=epochs)
print('> training time is %.4f minutes' % ((time.time() - starting_time)/60))

In [0]:
test_model(model, pub_testingset, x_pub_testing, y_pub_testing)

**Training and Evaluating the Model with DataAugmentation and EarlyStopping**

In [0]:
# 48x48 portrait image
input_image = Input(shape=(48, 48, 1), name='Input')

# conv, pooling layers + dropout
x = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_1')(input_image)
x = Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_2')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_3')(x)
x = Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_4')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv1_5')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='relu', name='conv1_6')(x)
x = MaxPooling2D(pool_size=(2, 2), name='pooling1')(x)
x = Dropout(rate=0.5, name='conv_dropout1')(x)

x = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_1')(x)
x = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_2')(x)
x = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_3')(x)
x = Conv2D(filters=256, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_4')(x)
x = Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu', name='conv2_5')(x)
x = Conv2D(filters=512, kernel_size=(3, 3), padding='valid', activation='relu', name='conv2_6')(x)
x = MaxPooling2D(pool_size=(2, 2), name='pooling2')(x)
x = Dropout(rate=0.5, name='conv_dropout2')(x)

# matrix ---> vector
x = Flatten(name='flatten')(x)

# FC layers + dropout
x = Dense(units=2048, activation='relu', name='fc1')(x)
x = Dropout(rate=0.5, name='fc_dropout1')(x)

x = Dense(units=2048, activation='relu', name='fc2')(x)
x = Dropout(rate=0.5, name='fc_dropout2')(x)

output_label = Dense(units=n_classes, activation='softmax', name='Output')(x)

# define model
model = Model(inputs=input_image, outputs=output_label, name='fer_cnn')

# print model summary
model.summary()

In [0]:
# declare learning rate, loss function, and model metric
loss = 'categorical_crossentropy'
lr = 0.0001
model.compile(loss=loss, optimizer=Adam(lr=lr), metrics=['accuracy'])

# Data augmentation
# based on https://keras.io/preprocessing/image/
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# EarlyStopping: stop training early to avoid overfitting
early_stopping_after_epochs = 10
early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping_after_epochs)

# ReduceLROnPlateau: reduce learning rate while training (new_lr = lr x factor)
reduce_lr_after_epochs = 10
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=reduce_lr_after_epochs, min_lr=0.0001)

***PublicTest***

In [0]:
# train the model
batch_size = 128
epochs = 100

# with DataAugmentation and EarlyStopping
starting_time = time.time()
history = model.fit_generator(datagen.flow(x_training, y_training, batch_size=batch_size),
                              steps_per_epoch=len(x_training) / batch_size,
                              validation_data=(x_pub_testing, y_pub_testing),
                              epochs=epochs,
                              callbacks=[early_stopping])
print('> training time is %.4f minutes' % ((time.time() - starting_time)/60))

In [0]:
test_model(model, pub_testingset, x_pub_testing, y_pub_testing)

***PrivateTest***

In [0]:
# train the model
batch_size = 128
epochs = 100

# with DataAugmentation and EarlyStopping
starting_time = time.time()
history = model.fit_generator(datagen.flow(x_training, y_training, batch_size=batch_size),
                              steps_per_epoch=len(x_training) / batch_size,
                              validation_data=(x_priv_testing, y_priv_testing),
                              epochs=epochs,
                              callbacks=[early_stopping])
print('> training time is %.4f minutes' % ((time.time() - starting_time)/60))

In [0]:
test_model(model, priv_testingset, x_priv_testing, y_priv_testing)